In [2]:
# autoreload modules
%load_ext autoreload
%autoreload 2

In [73]:
import json
import copy
import pandas as pd
from toolz.dicttoolz import assoc_in, merge, dissoc
from dotenv import dotenv_values
from pymongo import MongoClient
from pymongo.database import Database as MongoDatabase
from nmdc_runtime.api.core.metadata import load_changesheet, update_mongo_db

### load mongodb via env info

In [17]:
config = dotenv_values("../../.env.localhost")
config["MONGO_HOST"]

'mongodb://localhost:27018'

In [18]:
client = MongoClient(host=config["MONGO_HOST"], username=config["MONGO_USERNAME"], password=config["MONGO_PASSWORD"])
mongodb = client["nmdc_etl_staging"]

### creat temp database and temp_set collection for testing

In [125]:
if "temp_db" in client.list_database_names():
    client.drop_database("temp_db")

# create temp db with a dummy doc
temp_db = client["temp_db"]
temp_set = tempdb["temp_set"]
temp_set.insert_one({"id": 1, "value": "test"})

### function to copy change documents to temp db

In [126]:
def copy_change_sheet_to_temp_db(changeDf: pd.DataFrame, mongodb: MongoDatabase, temp_db: MongoDatabase):
    # remove dups
    df = changeDf[["group_id", "collection_name"]].drop_duplicates()
    for ix, id_val, collection_name in df.itertuples():
        # check if doc exists in temp db
        check_doc = temp_db[collection_name].find_one({"id": id_val}), "_id"
        if check_doc is not None:
            doc = dissoc(mongodb[collection_name].find_one({"id": id_val}), "_id")
            temp_db[collection_name].insert_one(doc)

### load change sheet and copy data to temp db

In [127]:
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)

In [128]:
# uncomment to see original change sheet
# pd.read_csv("data/changesheet-without-separator3.tsv", sep="\t", dtype="string").fillna('')

In [129]:
sheetDf = load_changesheet('data/changesheet-without-separator3.tsv', mongodb)
copy_change_sheet_to_temp_db(sheetDf, mongodb, temp_db)
sheetDf

,id,action,attribute,value,group_id,group_var,path,collection_name,class_name,prop_depth,prop_range,item_type
0,gold:Gs0110138,update,name,NEW STUDY NAME,gold:Gs0110138,,name,test.study_set,Study,1,string,
1,,update,ecosystem,NEW ECOSYSTEM,gold:Gs0110138,,ecosystem,test.study_set,Study,1,string,
2,,update,ecosystem_type,NEW ECOSYSTEM_TYPE,gold:Gs0110138,,ecosystem_type,test.study_set,Study,1,string,
3,,update,ecosystem_subtype,NEW ECOSYSTEM_SUBTYPE,gold:Gs0110138,,ecosystem_subtype,test.study_set,Study,1,string,
4,,update,doi,v1,gold:Gs0110138,v1,,test.study_set,Study,,,
5,v1,update,has_raw_value,NEW DOI,gold:Gs0110138,v1,doi.has_raw_value,test.study_set,Study,2,object:AttributeValue,
6,,update,principal_investigator,v2,gold:Gs0110138,v2,,test.study_set,Study,,,
7,v2,update,name,NEW PI NAME,gold:Gs0110138,v2,principal_investigator.name,test.study_set,Study,2,object:PersonValue,
8,v2,update,has_raw_value,NEW RAW NAME,gold:Gs0110138,v2,principal_investigator.has_raw_value,test.study_set,Study,2,object:PersonValue,
9,,update,description,NEW DESCRIPTION,gold:Gs0110138,,description,test.study_set,Study,1,string,


### update mongodb using change sheet

In [130]:
update_mongo_db(sheetDf, temp_db, print_update=False, print_data=False, print_query=False)

{'n': 12, 'nModified': 12, 'ok': 1.0}